In [1]:
import os
import pandas as pd
import pandas.io.sql as sqlio
from sqlalchemy import create_engine, text
from assistments.sdk.aws.manager import rds

In [3]:
# Create sql instance
manager = rds.Manager()
credentials = manager.get_db_credentials('rds-dev', 'tng')
url = f'postgresql://{credentials.username}:{credentials.password}@{credentials.host}:{credentials.port}/cas_core'
dev_engine = create_engine(url)
dev_conn = dev_engine.connect().execution_options(autocommit=True)

# Get school year data
school_year_data = []
for year in [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]:
    print(year)
    with open('sql/school_year_data_query.sql', 'rt') as fin:
        with open('sql/tmp.sql', 'wt') as fout:
            for line in fin:
                fout.write(line.replace('START_YEAR', str(year)).replace('END_YEAR', str(year + 1)))
    data = sqlio.read_sql_query(text(open('sql/tmp.sql', 'r').read()), dev_conn)
    os.remove('sql/tmp.sql')
    data['school_start_year'] = year
    data['school_end_year'] = year + 1
    school_year_data.append(data)
school_year_data = pd.concat(school_year_data, axis=0)
school_year_data.to_csv('csv/school_year_data.csv', index=False)

2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
